# Example of the SASRec training and inference stages
Note that all the given examples can be run without using PySpark, using only Pandas

In [1]:
# changing core directory
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
os.chdir('..')

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["OMP_NUM_THREADS"] = "4"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["KAGGLE_USERNAME"] = "recsysaccelerate"
os.environ["KAGGLE_KEY"] = "6363e91b656fea576c39e4f55dcc1d00"

In [3]:
import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
import torch

from replay.preprocessing.filters import MinCountFilter
from replay.metrics import OfflineMetrics, Recall, Precision, MAP, NDCG, HitRate, MRR
from replay.metrics.torch_metrics_builder import metrics_to_df
from replay.splitters import LastNSplitter, TimeSplitter
from replay.utils import get_spark_session
from replay.data import (
    FeatureHint,
    FeatureInfo,
    FeatureSchema,
    FeatureSource,
    FeatureType,
    Dataset,
)
from replay.models.nn.optimizer_utils import FatOptimizerFactory
from replay.models.nn.sequential.callbacks import (
    ValidationMetricsCallback,
    SparkPredictionCallback,
    PandasPredictionCallback, 
    TorchPredictionCallback,
    QueryEmbeddingsPredictionCallback
)
from replay.models.nn.sequential.postprocessors import RemoveSeenItems
from replay.data.nn import (
    SequenceTokenizer,
    SequentialDataset,
    TensorFeatureSource,
    TensorSchema,
    TensorFeatureInfo
)
from replay.models.nn.sequential import SasRec
from replay.models.nn.sequential.sasrec import (
    SasRecPredictionDataset,
    SasRecTrainingDataset,
    SasRecValidationDataset,
    SasRecPredictionBatch,
    SasRecModel
)

import pandas as pd

In [4]:
import random
import numpy as np

SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

torch.backends.cudnn.deterministic=True

## Getting a spark session

In [5]:
spark_session = get_spark_session()

/usr/local/lib/python3.11/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/28 07:39:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/28 07:39:42 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
24/11/28 07:39:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/11/28 07:39:42 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/11/28 07:39:42 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/11/28 07:39:42 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/11/28 07:39:42 WARN Utils: Se

## Prepare data
### Load raw movielens-1M interactions, item features and user features.
In the current implementation, the SASRec does not take into account the features of items or users. They are only used to get a complete list of users and items.

In [6]:
!pip install rs-datasets


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [7]:
from rs_datasets import MovieLens

In [8]:
movielens = MovieLens("1m")
interactions = movielens.ratings
user_features = movielens.users
item_features = movielens.items

In [9]:
interactions.head()

user_id  item_id  rating  timestamp
0        1     1193       5  978300760
1        1      661       3  978302109
2        1      914       3  978301968
3        1     3408       4  978300275
4        1     2355       5  978824291

In [10]:
user_features.head()

user_id gender  age  occupation zip_code
0        1      F    1          10    48067
1        2      M   56          16    70072
2        3      M   25          15    55117
3        4      M   45           7    02460
4        5      M   25          20    55455

In [11]:
item_features.head()

item_id                               title                        genres
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Drama
4        5  Father of the Bride Part II (1995)                        Comedy

In [12]:
interactions.describe()

user_id       item_id        rating     timestamp
count  1.000209e+06  1.000209e+06  1.000209e+06  1.000209e+06
mean   3.024512e+03  1.865540e+03  3.581564e+00  9.722437e+08
std    1.728413e+03  1.096041e+03  1.117102e+00  1.215256e+07
min    1.000000e+00  1.000000e+00  1.000000e+00  9.567039e+08
25%    1.506000e+03  1.030000e+03  3.000000e+00  9.653026e+08
50%    3.070000e+03  1.835000e+03  4.000000e+00  9.730180e+08
75%    4.476000e+03  2.770000e+03  4.000000e+00  9.752209e+08
max    6.040000e+03  3.952000e+03  5.000000e+00  1.046455e+09

In [13]:
interactions = MinCountFilter(
    num_entries=5,
    groupby_column='item_id',
).transform(interactions)

interactions = MinCountFilter(
    num_entries=3,
    groupby_column='user_id',
).transform(interactions)

In [14]:
interactions.describe()

user_id        item_id         rating     timestamp
count  999611.000000  999611.000000  999611.000000  9.996110e+05
mean     3024.576537    1865.439428       3.581974  9.722409e+08
std      1728.436705    1095.976336       1.116894  1.214827e+07
min         1.000000       1.000000       1.000000  9.567039e+08
25%      1506.000000    1030.000000       3.000000  9.653025e+08
50%      3070.000000    1835.000000       4.000000  9.730170e+08
75%      4477.000000    2770.000000       4.000000  9.752208e+08
max      6040.000000    3952.000000       5.000000  1.046455e+09

Removing duplicates in the timestamp column without changing the original items order where timestamp is the same

In [15]:
# interactions["timestamp"] = interactions["timestamp"].astype("int64")
# interactions = interactions.sort_values(by="timestamp")
# interactions["timestamp"] = interactions.groupby("user_id").cumcount()
# interactions

### Split interactions into the train, validation and test datasets using LastNSplitter

In [16]:
# splitter = LastNSplitter(
#     N=1,
#     divide_column="user_id",
#     query_column="user_id",
#     strategy="interactions",
# )

# raw_test_events, raw_test_gt = splitter.split(interactions)
# raw_validation_events, raw_validation_gt = splitter.split(raw_test_events)
# raw_train_events = raw_validation_events


splitter = TimeSplitter(
    time_threshold=0.1,
    drop_cold_users=True,
    drop_cold_items=True,
    item_column='item_id',
    query_column='user_id',
    timestamp_column='timestamp',
)

# train_events, validation_events, validation_gt, test_events, test_gt = (
#     _split_data(splitter, interactions)
# )

raw_test_events, raw_test_gt = splitter.split(interactions)
raw_validation_events, raw_validation_gt = splitter.split(raw_test_events)
raw_train_events = raw_validation_events

raw_test_gt = raw_test_gt[raw_test_gt['item_id'].isin(raw_train_events['item_id'])]
raw_test_gt = raw_test_gt[raw_test_gt['user_id'].isin(raw_train_events['user_id'])]

In [17]:
def test_splitting(events, gt, name=''):
    if events['timestamp'].max() > gt['timestamp'].min():
        print("Problem with time points in", name)
    if len(set(gt['user_id'].unique().tolist()) - set(events['user_id'].unique().tolist())) > 0:
        print("Problem with cold users in", name)
    if len(set(gt['item_id'].unique().tolist()) - set(events['item_id'].unique().tolist())) > 0:
        print("Problem with cold items in", name)


test_splitting(raw_train_events, raw_test_gt, "train events, test gt")
test_splitting(raw_train_events, raw_validation_gt, "train events, valid gt")

### Prepare FeatureSchema required to create Dataset

In [18]:
def prepare_feature_schema(is_ground_truth: bool) -> FeatureSchema:
    base_features = FeatureSchema(
        [
            FeatureInfo(
                column="user_id",
                feature_hint=FeatureHint.QUERY_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
            FeatureInfo(
                column="item_id",
                feature_hint=FeatureHint.ITEM_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
        ]
    )
    if is_ground_truth:
        return base_features

    all_features = base_features + FeatureSchema(
        [
            FeatureInfo(
                column="timestamp",
                feature_type=FeatureType.NUMERICAL,
                feature_hint=FeatureHint.TIMESTAMP,
            ),
        ]
    )
    return all_features

### Create Dataset for the training stage

In [19]:
user_features = None
item_features = None

In [20]:
train_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_train_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)

### Create Datasets (events and ground_truth) for the validation stage

In [21]:
validation_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_validation_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)
validation_gt = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=True),
    interactions=raw_validation_gt,
    check_consistency=True,
    categorical_encoded=False,
)

### Create Datasets (events and ground_truth) for the testing stage

In [22]:
test_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_test_events,
    query_features=user_features,
    item_features=item_features,
    check_consistency=True,
    categorical_encoded=False,
)
test_gt = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=True),
    interactions=raw_test_gt,
    check_consistency=True,
    categorical_encoded=False,
)

### Create the tensor schema
A schema shows the correspondence of columns from the source dataset with the internal representation of tensors inside the model

In [23]:
ITEM_FEATURE_NAME = "item_id_seq"

tensor_schema = TensorSchema(
    TensorFeatureInfo(
        name=ITEM_FEATURE_NAME,
        is_seq=True,
        feature_type=FeatureType.CATEGORICAL,
        feature_sources=[TensorFeatureSource(FeatureSource.INTERACTIONS, train_dataset.feature_schema.item_id_column)],
        feature_hint=FeatureHint.ITEM_ID,
    )
)

### Create sequential datasets using SequenceTokenizer
The SequentialDataset internally store data in the form of sequences of items sorted by increasing interaction time (timestamp). A SequenceTokenizer is used to convert to this format. In addition, the SequenceTokenizer encodes all categorical columns from the source dataset and stores mapping inside itself.
SequentialDataset.keep_common_query_ids is used to leave only sequences from the same users

In [24]:
tokenizer = SequenceTokenizer(tensor_schema, allow_collect_to_master=True, handle_unknown_rule="drop")
tokenizer.fit(train_dataset)

sequential_train_dataset = tokenizer.transform(train_dataset)

sequential_validation_dataset = tokenizer.transform(validation_dataset)
sequential_validation_gt = tokenizer.transform(validation_gt, [tensor_schema.item_id_feature_name])

sequential_validation_dataset, sequential_validation_gt = SequentialDataset.keep_common_query_ids(
    sequential_validation_dataset, sequential_validation_gt
)

In [25]:
test_query_ids = test_gt.query_ids
test_query_ids_np = tokenizer.query_id_encoder.transform(test_query_ids)["user_id"].values
sequential_test_dataset = tokenizer.transform(test_dataset).filter_by_query_id(test_query_ids_np)

You can get the user and item mapping and inverse mapping as follows

In [26]:
print(tokenizer.query_id_encoder.mapping, tokenizer.query_id_encoder.inverse_mapping)
print(tokenizer.item_id_encoder.mapping, tokenizer.item_id_encoder.inverse_mapping)

print(len(tokenizer.query_id_encoder.mapping["user_id"]), len(tokenizer.item_id_encoder.mapping['item_id']))

{'user_id': {6040: 0, 6039: 1, 6038: 2, 6037: 3, 6036: 4, 6035: 5, 6034: 6, 6033: 7, 6032: 8, 6031: 9, 6030: 10, 6029: 11, 6028: 12, 6027: 13, 6026: 14, 6025: 15, 6024: 16, 6023: 17, 6022: 18, 6021: 19, 6020: 20, 6019: 21, 6018: 22, 6017: 23, 6016: 24, 6015: 25, 6014: 26, 6013: 27, 6012: 28, 6011: 29, 6010: 30, 6009: 31, 6007: 32, 6008: 33, 6006: 34, 6005: 35, 6004: 36, 6003: 37, 6002: 38, 6001: 39, 6000: 40, 5999: 41, 5998: 42, 5997: 43, 5996: 44, 5995: 45, 5994: 46, 5993: 47, 5992: 48, 5991: 49, 5990: 50, 5989: 51, 5988: 52, 5987: 53, 5986: 54, 5984: 55, 5983: 56, 5982: 57, 5981: 58, 5979: 59, 5980: 60, 5978: 61, 5977: 62, 5976: 63, 5975: 64, 5974: 65, 5973: 66, 5972: 67, 5971: 68, 5970: 69, 5969: 70, 5968: 71, 5967: 72, 5966: 73, 5965: 74, 5964: 75, 5963: 76, 5962: 77, 5961: 78, 5960: 79, 5959: 80, 5958: 81, 5957: 82, 5956: 83, 5955: 84, 5954: 85, 5953: 86, 5952: 87, 5951: 88, 5950: 89, 5948: 90, 5947: 91, 5946: 92, 5945: 93, 5944: 94, 5943: 95, 5942: 96, 5941: 97, 5940: 98, 5939: 9

In [27]:
tokenizer.item_id_encoder.mapping['item_id'][1961]

4

## Train model
### Create SASRec model instance and run the training stage using lightning
After each epoch validation metrics are shown. You can change the list of validation metrics in ValidationMetricsCallback
The model is determined to be the best and is saved if the metric updates its maximum during validation (see the ModelCheckpoint)

In [ ]:
MAX_SEQ_LEN = 200
BATCH_SIZE = 512
NUM_WORKERS = 4

model = SasRec(
    tensor_schema,
    block_count=2,
    head_count=2,
    max_seq_len=MAX_SEQ_LEN,
    hidden_size=300,
    dropout_rate=0.5,
    optimizer_factory=FatOptimizerFactory(learning_rate=0.001),
)

csv_logger = CSVLogger(save_dir=".logs/train", name="SASRec_example")

checkpoint_callback = ModelCheckpoint(
    dirpath=".checkpoints",
    save_top_k=1,
    verbose=True,
    # if you use multiple dataloaders, then add the serial number of the dataloader to the suffix of the metric name.
    # For example,"recall@10/dataloader_idx_0"
    monitor="recall@10",
    mode="max",
)

validation_metrics_callback = ValidationMetricsCallback(
    metrics=["map", "ndcg", "recall"],
    ks=[1, 5, 10, 20],
    item_count=train_dataset.item_count,
    postprocessors=[RemoveSeenItems(sequential_validation_dataset)]
)

trainer = L.Trainer(
    max_epochs=100,
    callbacks=[checkpoint_callback, validation_metrics_callback],
    logger=csv_logger,
)

train_dataloader = DataLoader(
    dataset=SasRecTrainingDataset(
        sequential_train_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
)

validation_dataloader = DataLoader(
    dataset=SasRecValidationDataset(
        sequential_validation_dataset,
        sequential_validation_gt,
        sequential_train_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

trainer.fit(
    model,
    train_dataloaders=train_dataloader,
    val_dataloaders=validation_dataloader,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L40') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/usr/local/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:653: Checkpoint directory /root/RePlay-Accelerated/.checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type             | Params
--------------------------------------------
0 | _model | SasRecModel      | 2.2 M 
1 | _loss  | CrossEntropyLoss | 0     
--------------------------------------------
2.2 M     Trainable params
0         Non-trainable params
2.2 M     To

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

k              1        10        20         5
map     0.018405  0.005376  0.004157  0.007485
ndcg    0.018405  0.014625  0.014979  0.014758
recall  0.000425  0.002585  0.004329  0.000918



/usr/local/lib/python3.11/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (11) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

The path to the best model is saved inside checkpoint_callback

In [ ]:
best_model = SasRec.load_from_checkpoint(checkpoint_callback.best_model_path)

## Inference stage
### Prepare Dataloader and logger

In [ ]:
prediction_dataloader = DataLoader(
    dataset=SasRecPredictionDataset(
        sequential_test_dataset,
        max_sequence_length=MAX_SEQ_LEN,
    ),
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

csv_logger = CSVLogger(save_dir=".logs/test", name="SASRec_example")

### Run inference
You can get the recommendations in three formats: PySpark DataFrame, Pandas DataFrame, PyTorch tensors. Each of the types corresponds a callback
You can filter the results using postprocessors strategy. For example the RemoveSeenItems postprocessor is filtering out the items that already have been seen in test dataset
You don't need to use all three callbacks. This is shown only for example

Also, you can get user embeddings, that were used to perform predictions, using `get_query_embedding` method inside SasRecModel or `QueryEmbeddingsPredictionCallback` for lightning module.

In [ ]:
TOPK = [1, 10, 20, 100]

postprocessors = [RemoveSeenItems(sequential_test_dataset)]

spark_prediction_callback = SparkPredictionCallback(
    spark_session=spark_session,
    top_k=max(TOPK),
    query_column="user_id",
    item_column="item_id",
    rating_column="score",
    postprocessors=postprocessors,
)

pandas_prediction_callback = PandasPredictionCallback(
    top_k=max(TOPK),
    query_column="user_id",
    item_column="item_id",
    rating_column="score",
    postprocessors=postprocessors,
)

torch_prediction_callback = TorchPredictionCallback(
    top_k=max(TOPK),
    postprocessors=postprocessors,
)

query_embeddings_callback = QueryEmbeddingsPredictionCallback()

trainer = L.Trainer(
    callbacks=[
        spark_prediction_callback,
        pandas_prediction_callback,
        torch_prediction_callback,
        query_embeddings_callback
    ], 
    logger=csv_logger, 
    inference_mode=True
)
trainer.predict(best_model, dataloaders=prediction_dataloader, return_predictions=False)

spark_res = spark_prediction_callback.get_result()
pandas_res = pandas_prediction_callback.get_result()
torch_user_ids, torch_item_ids, torch_scores = torch_prediction_callback.get_result()
user_embeddings = query_embeddings_callback.get_result()

In [ ]:
spark_res.show()

+-------+-------+------------------+
|user_id|item_id|             score|
+-------+-------+------------------+
|      0|   2012| 8.138867378234863|
|      0|   1642|  8.09422492980957|
|      0|    360| 8.087743759155273|
|      0|   2325| 7.976056098937988|
|      0|     47|  7.48519229888916|
|      0|   2020| 7.362086296081543|
|      0|   2016| 7.105367660522461|
|      0|   2009| 7.090343475341797|
|      0|   1020|  7.08748197555542|
|      0|   2014| 6.992861747741699|
|      0|     12| 6.892394542694092|
|      0|   1382| 6.864995956420898|
|      0|    612| 6.860413074493408|
|      0|   2027|  6.73327112197876|
|      0|   3088| 6.721376419067383|
|      0|   2018| 6.611262321472168|
|      0|   1012| 6.602219581604004|
|      0|   1019|6.5607428550720215|
|      0|   3090| 6.448214530944824|
|      0|   2965| 6.409302711486816|
+-------+-------+------------------+
only showing top 20 rows



In [ ]:
pandas_res

user_id item_id     score
0           0    2012  8.138867
0           0    1642  8.094225
0           0     360  8.087744
0           0    2325  7.976056
0           0      47  7.485192
...       ...     ...       ...
6039     6039    1251  3.320609
6039     6039    2174  3.301445
6039     6039    3197  3.286182
6039     6039    3141  3.280689
6039     6039    2849    3.2665

[604000 rows x 3 columns]

In [ ]:
print(torch_user_ids[0], torch_item_ids[0], torch_scores[0])

tensor(0) tensor([2012, 1642,  360, 2325,   47, 2020, 2016, 2009, 1020, 2014,   12, 1382,
         612, 2027, 3088, 2018, 1012, 1019, 3090, 2965,  547, 2068, 2021, 1990,
        2073, 2315, 2072,  700,  105, 1050,  582, 1346, 2011, 2918,  310, 1743,
        1977,  314, 3546, 1058,  974, 1010, 1850, 2023,  358, 1979,  259, 2631,
         667,   33, 2030, 2070, 2013, 1002, 2640, 3328, 2807,  592, 1964, 3144,
        2830, 2985, 3327, 3329, 1262, 1017, 1547,  476,    1, 2285, 1459, 1664,
        1018,   59, 2692, 1000, 1936, 1851, 3218, 3414, 1539,  826, 1011, 3106,
         999,  791, 2092, 1004, 2047,  236, 2731, 3676, 2677, 3682,  156, 2071,
        1008,  363, 1945, 2777]) tensor([8.1389, 8.0942, 8.0877, 7.9761, 7.4852, 7.3621, 7.1054, 7.0903, 7.0875,
        6.9929, 6.8924, 6.8650, 6.8604, 6.7333, 6.7214, 6.6113, 6.6022, 6.5607,
        6.4482, 6.4093, 6.3793, 6.3753, 6.2767, 6.2201, 6.2057, 6.1955, 6.1510,
        6.1112, 6.0373, 6.0356, 5.9954, 5.9927, 5.9892, 5.9694, 5.9151, 5.876

Suppose we want to get the recomendations in PySpark format. 
Let's get the inverse representation of labels using inverse_transform method.

Note that the reverse representation can only be obtained for PySpark and Pandas formats. When working with PyTorch tensors, the reverse representation must be done manually

In [ ]:
recommendations = tokenizer.query_and_item_id_encoder.inverse_transform(spark_res)

In [ ]:
recommendations.show()

+------------------+-------+-------+
|             score|user_id|item_id|
+------------------+-------+-------+
| 8.138867378234863|      1|   2081|
|  8.09422492980957|      1|   1688|
| 8.087743759155273|      1|    364|
| 7.976056098937988|      1|   2394|
|  7.48519229888916|      1|     48|
| 7.362086296081543|      1|   2089|
| 7.105367660522461|      1|   2085|
| 7.090343475341797|      1|   2078|
|  7.08748197555542|      1|   1033|
| 6.992861747741699|      1|   2083|
| 6.892394542694092|      1|     13|
| 6.864995956420898|      1|   1405|
| 6.860413074493408|      1|    616|
|  6.73327112197876|      1|   2096|
| 6.721376419067383|      1|   3157|
| 6.611262321472168|      1|   2087|
| 6.602219581604004|      1|   1025|
|6.5607428550720215|      1|   1032|
| 6.448214530944824|      1|   3159|
| 6.409302711486816|      1|   3034|
+------------------+-------+-------+
only showing top 20 rows



### Calculating metrics

In [ ]:
init_args = {"query_column": "user_id", "rating_column": "score"}

In [ ]:
result_metrics = OfflineMetrics(
    [Recall(TOPK), Precision(TOPK), MAP(TOPK), NDCG(TOPK), MRR(TOPK), HitRate(TOPK)], **init_args
)(recommendations.toPandas(), raw_test_gt)

In [ ]:
metrics_to_df(result_metrics)

k              1         10        20        100
HitRate    0.06904  0.296026  0.415397  0.680464
MAP        0.06904  0.126483  0.134644  0.141227
MRR        0.06904  0.126483  0.134644  0.141227
NDCG       0.06904  0.165998  0.196025  0.244784
Precision  0.06904  0.029603  0.020770  0.006805
Recall     0.06904  0.296026  0.415397  0.680464

### User embeddings

Got 6040 x 300 user embeddings, because among all 12 batches: 

11 batches contains 512 samples

1 batch contains 408 left samples

11 * 512 + 408 == 6040

In [ ]:
user_embeddings

tensor([[ 0.3704,  1.9303,  1.9161,  ..., -0.1143,  1.5672, -0.9027],
        [-1.8589, -2.6053,  0.1639,  ..., -0.4250, -2.5378, -0.8178],
        [-0.0877,  0.3147,  2.4707,  ..., -0.7766, -0.8095, -1.2838],
        ...,
        [-0.5727,  0.1162,  1.5556,  ..., -1.8936,  1.4968, -1.3920],
        [-0.3458, -1.6828, -0.5697,  ..., -1.6742,  3.2049,  0.6597],
        [-1.2831,  0.7636,  1.0783,  ..., -1.1996, -0.0189,  1.4718]],
       device='cuda:0')

In [ ]:
user_embeddings.shape

torch.Size([6040, 300])

You can access user embeddings directly with `SasRecModel` class

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

core_model = SasRecModel(
    tensor_schema,
    num_blocks=2,
    num_heads=2,
    max_len=MAX_SEQ_LEN,
    hidden_size=300,
    dropout=0.5
)
core_model.eval()
core_model = core_model.to(device)

# Get first batch of data 
data = next(iter(prediction_dataloader))
tensor_map, padding_mask = data.features, data.padding_mask

# Ensure everything is on the same device
padding_mask = padding_mask.to(device)
tensor_map["item_id_seq"] = tensor_map["item_id_seq"].to(device)

# Get user embeddings
user_embeddings_batch = core_model.get_query_embeddings(tensor_map, padding_mask)
user_embeddings_batch

tensor([[-4.3240e-01, -1.0684e-01, -8.1389e-02,  ...,  6.6867e-01,
          4.7710e-01, -1.7522e+00],
        [ 7.8765e-02,  8.0354e-01, -9.5730e-01,  ..., -1.9753e+00,
          1.4708e+00, -7.7084e-01],
        [ 1.0037e+00,  6.5928e-01, -1.4148e+00,  ...,  1.0593e+00,
         -7.4484e-01, -2.7631e-01],
        ...,
        [-1.4800e+00, -1.9950e-01, -3.6084e-01,  ...,  1.5323e+00,
          2.6428e-01, -8.7467e-01],
        [ 1.1612e+00, -6.3325e-04,  4.0772e-01,  ...,  2.5503e-01,
         -3.4220e-01,  5.4465e-01],
        [-6.3809e-01,  1.0407e+00, -2.1736e+00,  ...,  8.7484e-01,
          1.4989e+00, -1.1730e+00]], device='cuda:0', grad_fn=<SliceBackward0>)

In [ ]:
user_embeddings_batch.shape

torch.Size([512, 300])

### Item embeddings

`get_all_embeddings()` method in transformers can be used to get copies of all embeddings that are presented in model as a dict.

In [ ]:
all_embeddings = best_model.get_all_embeddings()
all_embeddings

{'item_embedding': tensor([[-0.0238,  0.0208, -0.0037,  ...,  0.0340,  0.0224, -0.0369],
         [-0.0103,  0.0099,  0.0112,  ...,  0.0112,  0.0255, -0.0349],
         [ 0.0090, -0.0228, -0.0544,  ...,  0.0024,  0.0438,  0.0060],
         ...,
         [-0.0220,  0.0150, -0.0027,  ...,  0.0562,  0.0170, -0.0025],
         [ 0.0147, -0.0004,  0.0214,  ..., -0.0098, -0.0017,  0.0299],
         [-0.0095,  0.0142, -0.0030,  ...,  0.0435, -0.0034,  0.0077]]),
 'positional_embedding': tensor([[ 0.1256,  0.0453, -0.0112,  ..., -0.1254,  0.0047, -0.0691],
         [-0.0208, -0.0014,  0.0146,  ...,  0.0680, -0.0909, -0.0365],
         [-0.0839,  0.0381,  0.0081,  ..., -0.0139, -0.0645, -0.0350],
         ...,
         [ 0.0322,  0.0308, -0.0525,  ...,  0.0426,  0.0454,  0.0101],
         [-0.0335, -0.0006,  0.0082,  ..., -0.0202, -0.0435, -0.0786],
         [-0.0537, -0.0550,  0.1043,  ...,  0.0992,  0.0194,  0.0321]])}

You can access item embeddings from this dictionary

In [ ]:
item_embeddings = all_embeddings["item_embedding"]
item_embeddings

tensor([[-0.0238,  0.0208, -0.0037,  ...,  0.0340,  0.0224, -0.0369],
        [-0.0103,  0.0099,  0.0112,  ...,  0.0112,  0.0255, -0.0349],
        [ 0.0090, -0.0228, -0.0544,  ...,  0.0024,  0.0438,  0.0060],
        ...,
        [-0.0220,  0.0150, -0.0027,  ...,  0.0562,  0.0170, -0.0025],
        [ 0.0147, -0.0004,  0.0214,  ..., -0.0098, -0.0017,  0.0299],
        [-0.0095,  0.0142, -0.0030,  ...,  0.0435, -0.0034,  0.0077]])

Item embeddings shape is (N_ITEMS, HIDDEN_SIZE)

In [ ]:
item_embeddings.shape

torch.Size([3883, 300])

Ensure we got correct dimension and ensure we got the copy of tensor

In [ ]:
assert item_embeddings.shape[0] == len(tokenizer.item_id_encoder.mapping["item_id"])
assert id(item_embeddings) != id(best_model._model.item_embedder.item_emb.weight.data)

For example we observe one new item id in our training data. We can easily expand our item embedder by one element

In order to expand item embeddings by new size `set_item_embeddings_by_size` method is applied

In [ ]:
best_model.set_item_embeddings_by_size(item_embeddings.shape[0] + 1)

Now our new item embeddings have one extra embedding

In [ ]:
new_size = best_model.get_all_embeddings()["item_embedding"].shape[0]
old_size = item_embeddings.shape[0]

assert new_size == old_size + 1

Alternatively, we can pass our item embeddings that replace the existing ones by calling `set_item_embeddings_by_tensor`.

If tensor contains new items, they will be added to item embedder.

In [ ]:
new_embeddings_weights = torch.rand((new_size + 1, 300))    # randint used for example only

best_model.set_item_embeddings_by_tensor(new_embeddings_weights)

At the moment we expanded our item embeddings by one more item and replace weights by passing `new_embeddings_weights`

In [ ]:
old_size = new_size
new_size = best_model.get_all_embeddings()["item_embedding"].shape[0]

assert new_size == old_size + 1

Similarly, we can append tensor for only new items with no replace for existing by calling `append_item_embeddings`

In [ ]:
new_item_weights = torch.rand((1, 300))    # randint used for example only

best_model.append_item_embeddings(new_item_weights)

We passed one new example and its weights to item embeddings, thus expanded our vocabulary by one item again

In [ ]:
old_size = new_size
new_size = best_model.get_all_embeddings()["item_embedding"].shape[0]

assert new_size == old_size + 1

## Example of launching an inference for a single user without using a trainer (in order to speed up)
An example for the production of an online script

Let's assume that the user's sequence consisted of a sequence of items [1, 2, 3, 4, 5]. 
Сreate a padding mask corresponding to the sequence of items.

It is important to take only the latest MAX_SEQ_LEN or less items.

In [ ]:
item_sequence = torch.arange(1, 5).unsqueeze(0)[:, -MAX_SEQ_LEN:]
padding_mask = torch.ones_like(item_sequence, dtype=torch.bool)
sequence_item_count = item_sequence.shape[1]

### Wrapping created tensors in the SasRecPredictionBatch entity

In [ ]:
batch = SasRecPredictionBatch(
    query_id=torch.arange(0, item_sequence.shape[0], 1).long(),
    padding_mask=padding_mask.bool(),
    features={ITEM_FEATURE_NAME: item_sequence.long()}
)

### Run predict step of the SasRec and get scores from the model

In [ ]:
with torch.no_grad():
    scores = best_model.predict_step(batch, 0)
scores

tensor([[ 2.7048,  2.2736,  2.1556,  ..., -0.2144, -1.3674,  1.9593]])

### Getting three items with the highest score

In [ ]:
torch.topk(scores, k=3).indices

tensor([[   4, 2013, 3032]])